In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
def link2soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.text, 'html.parser')

In [17]:
# First we need to crawl to get the links for the best card sections

main_soup = link2soup("https://www.nerdwallet.com/credit-cards")

hrefs = [
    a["href"]
    for li in main_soup.select("ul#l3ListWrapper-0-0 > li.l3ListItem._3DKUn-t")
    if (a := li.find("a", href=True))
]

In [104]:
# very rudimentary class for data storage

class CreditCard:
    def __init__(self, name, annual_fee, welcome_amount, apr, rewards):
        self.name = name
        self.annual_fee = annual_fee
        self.welcome_amount = welcome_amount
        self.apr = apr
        self.rewards = rewards
        
    def __repr__(self):
        return f"CreditCard(name = {self.name},\n annual_fee = {self.annual_fee},\n welcome_amount = {self.welcome_amount},\n apr = {self.apr},\n rewards = {self.rewards})"

In [123]:
cards = []

for href in hrefs:
    best_soup = link2soup(href)
    blocks = best_soup.select("div.MuiBox-root.css-1hlkqtw")
    for block in blocks:
        name = block.select("h3.MuiTypography-root.MuiTypography-body1.css-monr6r")[0].text
        table = block.select("div.MuiGrid-root.MuiGrid-container.MuiGrid-direction-xs-row.css-7zk183")
        first_row = table[0].select("div.MuiGrid-root.MuiGrid-direction-xs-row.css-43v8ft")
            
        annual_fee = None
        welcome_amount = None
        apr = None
        rewards = []
            
        for entry in first_row:
            if entry.select("div.MuiBox-root.css-dayuin")[0].text == "Annual fee":
                annual_fee = entry.select("div.MuiBox-root.css-1ffk1vi")[0].text
            elif entry.select("div.MuiBox-root.css-dayuin")[0].text == "Intro offer":
                welcome_amount = entry.select("div.MuiBox-root.css-1ffk1vi")[0].text
        other_rows = table[0].select("div.MuiGrid-root.MuiGrid-container.MuiGrid-direction-xs-row.css-1a97p8s")
        for row in other_rows:
            if row.select("div.MuiBox-root.css-dayuin")[0].text == "Regular APR":
                apr = row.select("div.MuiBox-root.css-1ffk1vi")[0].text
            
        reward_block = block.select("details.MuiBox-root.css-171u67s")
        reward_block = reward_block[0].select("div.MuiBox-root.css-1wcs7fe")
        if reward_block != []:
            rewards = [
                div.text+" "+div.find_next_sibling("span").text for div in reward_block[0].select("div.MuiBox-root")
            ]
        credit_card = CreditCard(name,annual_fee,welcome_amount,apr,rewards)
        cards.append(credit_card)

In [125]:
len(cards)

81

In [126]:
len(hrefs)

10